Hugging face transfomers ,datsets,evaluate like BLEU

In [ ]:
# Uninstall W&B to avoid errors
!pip uninstall -y wandb

# Install Hugging Face libraries
!pip install -U transformers datasets evaluate -q

In [ ]:
import os

# Disable wandb before it's ever used
os.environ["WANDB_DISABLED"] = "true"

# Optional but good practice
os.environ["WANDB_MODE"] = "offline"


In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
import pandas as pd

# Load your CSV files
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test.csv")

# Combine Title and Description into a single text column
train_df["text"] = train_df["Title"] + " " + train_df["Description"]
test_df["text"] = test_df["Title"] + " " + test_df["Description"]

# Convert class index from 1-4 to 0-3
train_df["label"] = train_df["Class Index"] - 1
test_df["label"] = test_df["Class Index"] - 1

# Keep only necessary columns
train_df = train_df[["text", "label"]]
test_df = test_df[["text", "label"]]


In [ ]:
train_df.sample(5)

In [ ]:
from datasets import Dataset

# Convert to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
from datasets import DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})


In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

encoded_dataset = dataset.map(tokenize, batched=True)
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


In [ ]:
encoded_dataset["train"][0]

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import evaluate
import numpy as np

# Load model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

# Define metrics
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training setup
# Ensure transformers is up-to-date before this
training_args = TrainingArguments(
    output_dir="./results",
    # 'evaluation_strategy' was renamed to 'eval_strategy' in newer versions
    eval_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=50,
    weight_decay=0.01
)

# Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    compute_metrics=compute_metrics
)

In [ ]:
for param in model.distilbert.parameters():   #transfer learning so freezing the layers of distillbert
    param.requires_grad = False

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
import torch

# Set the device (automatically handles GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the right device

# Updated predict function
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to same device
    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = outputs.logits.argmax(dim=1).item()
    return predicted_class


text = "Virat kohli won the ipl trophy"
print("Predicted class index:", predict(text))



In [ ]:
label_map = {0: "World", 1: "Sports", 2: "Business", 3: "Sci/Tech"}
print("Predicted Label:", label_map[predict(text)])
